In [ ]:
import cv2
import numpy as np

# Read the image outside of the function to avoid reading it multiple times
image = cv2.imread("image.jpg")

def image_correction(src_points, dst_points):
    # Define original and destination points
    src_pts = np.array(src_points, dtype=np.float32)
    dst_pts = np.array(dst_points, dtype=np.float32)

    # Calculate perspective transformation matrix
    M = cv2.getPerspectiveTransform(src_pts, dst_pts)

    # Apply perspective transformation
    corrected_image = cv2.warpPerspective(image, M, (image.shape[1], image.shape[0]))

    # Display original and corrected images
    cv2.imshow("Original Image", image)
    cv2.imshow("Corrected Image", corrected_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Example usage
src_points = [(x1, y1), (x2, y2), (x3, y3), (x4, y4)]  # Original image points
dst_points = [(u1, v1), (u2, v2), (u3, v3), (u4, v4)]  # Destination points

image_correction(src_points, dst_points)



In [ ]:
import cv2
import numpy as np
import json

def create_mask(image, json_string):
    # 解析json字符串
    points_info = json.loads(json_string)
    points = np.array(points_info['points'])  # 假设points字段包含电视机的四个顶点

    # 创建一个全0的mask，大小和输入图片一致
    mask = np.zeros(image.shape[:2], dtype=np.uint8)

    # 使用cv2.fillPoly方法填充mask
    cv2.fillPoly(mask, [points], 255)

    return mask

import torch
from torchvision import models

def create_model(num_classes):
    model = models.segmentation.fcn_resnet50(weights='DEFAULT')

    model.classifier[4] = torch.nn.Conv2d(512, num_classes, kernel_size=(1, 1), stride=(1, 1))

    return model

model = create_model(2)


In [ ]:
from torchinfo import summary
batch_size = 1
summary(model, input_size=(batch_size, 3, 224, 224))

In [ ]:

def train_model(model, dataloader, device, optimizer, criterion, num_epochs):
    model = model.to(device)
    for epoch in range(num_epochs):
        for i, (inputs, labels) in enumerate(dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)

            # 前向传播
            outputs = model(inputs)['out']

            # 计算损失
            loss = criterion(outputs, labels)

            # 反向传播
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()